In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paper-tilt-brush/Looking_through_the_sphere_Illustration.pdf
/kaggle/input/rendering/Physically_Based_Rendering(PBRT)_3rd_Ed(2016).pdf
/kaggle/input/research-paper/seg of MI with ADMBNN and gausian ope.pdf


# We are using RAG architectures which basically consist two components: indexing and retrivel generation.

Installing Dependencies:

In [3]:
!pip install langchain
!pip install cohere
!pip install PyPDF2
!pip install faiss-cpu
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the 

In [4]:
from PyPDF2 import PdfReader
from langchain_community.embeddings import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

# 1. Indexing component of RAG

Setting Environment variable using Conhere api key and loading PDF file using pdfreader

In [5]:
os.environ["COHERE_API_KEY"] = "fL4XmW6RA16o6AuV5eZlCZd0YebYD0sQXQsersSH"
pdfreader = PdfReader('/kaggle/input/paper-tilt-brush/Looking_through_the_sphere_Illustration.pdf')

Extracting Text from PDF and splittering into smaller chunks

In [6]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [7]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

Downloading embeddings to index and atttaching FAISS vector store for future searching 

In [8]:
# Download embeddings from Cohere
embeddings = CohereEmbeddings()
document_search = FAISS.from_texts(texts, embeddings)

# 2. Retrival and generating component of RAG

Chaining is basically retriving data through searching on vectorestore with relevant text from input

In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import Cohere
chain = load_qa_chain(Cohere(), chain_type="stuff")

# Example (1): Giving query direclty to chain.run()

In [10]:
query = "give summary "
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The intersection of art and technology has given rise to new methods of visual communication through illustration, especially with the adoption of digital media and Virtual Reality (VR). VR illustration utilises special equipment such as head-mounted displays and sensors to create immersive experiences for users, allowing them to interact with 3D environments. While computers are tools for implementing design ideas, the responsibility for creating appealing VR experiences ultimately lies with designers who must adapt their practices to cater to this new medium. '

# Example(2): Using Gradio interface which takes prompt as input and give answer into output window.

In [12]:
import gradio as gr
from gradio.components import Textbox

# Defining answer function
def Answer(prompt):
    return chain.run(input_documents=docs, question=prompt)


# Creating the interface using Gradio
iface = gr.Interface(
    fn=Answer,
    inputs=Textbox(lines=2, label="Enter a prompt"),
    outputs=Textbox(lines=8,label="Answer"),
    live=True,
    title="PDF query search and QnA Demo",
    description="Enter a prompt."
)

iface.launch()


Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://5d1ee005b29eb56b1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
